# Introducción

El siguiente informa abarca estudios en torno a la educación primaria y secundaria en Chile, y tiene como principal objetivo caracterizar la relación entre el promedio general y distintas variables, como la condición de ruralidad, región geográfica, género, tipo de dependencia. Estas variables están presentes en el mismo dataset otorgado por el MINEDUC, que contiene gran información acerda de tres millones de estudiantes. Como es bien sabido, Chile es un país muy desigual y sumamente segregado en varios ámbitos. El objetivo de este proyecto es buscar patrones en los datos que puedan reflejar estas desigualdades.

El informe contiene análisis y síntesis de distintas observaciones en torno a los estudios y visualizaciones del comportamiento de estas variables.

# Exploración


In [55]:
import pandas as pd
import numpy as np

rendimiento = pd.read_csv("datos/mineduc/rendimiento_escolar/2020.csv",
                          sep=";", decimal=",",
                          dtype={
                              "NOM_RBD":"category",
                              "NOM_REG_RBD_A":"category",
                              "NOM_COM_RBD":"category",
                              "NOM_DEPROV_RBD":"category",
                              "LET_CUR":"category",
                              "GEN_ALU":"category",
                              "NOM_COM_ALU":"category",
                              "SIT_FIN":"category",
                              "SIT_FIN_R":"category",
                              "RURAL_RBD":"category",
                              "COD_DEPE":"category",
                              "ESTADO_ESTAB":"category"},
                          converters={"EDAD_ALU": lambda s : int(s) if s != ' ' else 0})

In [56]:
# cambio de nombre para algunas categorias
rendimiento.GEN_ALU.cat.categories = ["HOMBRE", "MUJER", "S/I"]

rendimiento.RURAL_RBD.cat.categories = ["URBANO", "RURAL"]

rendimiento.COD_DEPE = rendimiento.COD_DEPE.cat.rename_categories(
    {"1":"CORP_MUNICIPAL", "2":"MUNICIPAL_DAEM", "3":"PARTICULAR_SUBVEN",
     "4":"PARTICULAR_PAGADO", "5":"CORP_ADMIN_DELEGD", "6":"SERVI_LOCAL_EDUCA"})

rendimiento.ESTADO_ESTAB = rendimiento.ESTADO_ESTAB.cat.rename_categories(
    {"1":"FUNCIONANDO","2":"RECESO", "3":"CERRADO"}
)

rendimiento.SIT_FIN = rendimiento.SIT_FIN.cat.rename_categories(
    {" ":"SIN_INFO", "P":"PROMOVIDO", "R":"REPROBADO", "Y":"RETIRADO"}
)

rendimiento.SIT_FIN_R = rendimiento.SIT_FIN_R.cat.rename_categories (
    {" ":"SIN_INFO", "P":"PROMOVIDO", "R":"REPROBADO", "Y":"RETIRADO", "T":"TRASLADADO"}
)
pass
rendimiento.SIT_FIN_R

0          PROMOVIDO
1          PROMOVIDO
2           RETIRADO
3          PROMOVIDO
4          PROMOVIDO
             ...    
3164529    PROMOVIDO
3164530    PROMOVIDO
3164531    PROMOVIDO
3164532    PROMOVIDO
3164533    PROMOVIDO
Name: SIT_FIN_R, Length: 3164534, dtype: category
Categories (5, object): ['PROMOVIDO', 'REPROBADO', 'TRASLADADO', 'RETIRADO', 'SIN_INFO']

In [3]:
rendimiento.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3164534 entries, 0 to 3164533
Data columns (total 37 columns):
 #   Column          Dtype   
---  ------          -----   
 0   AGNO            int64   
 1   RBD             int64   
 2   DGV_RBD         int64   
 3   NOM_RBD         category
 4   COD_REG_RBD     int64   
 5   NOM_REG_RBD_A   category
 6   COD_PRO_RBD     int64   
 7   COD_COM_RBD     int64   
 8   NOM_COM_RBD     category
 9   COD_DEPROV_RBD  int64   
 10  NOM_DEPROV_RBD  category
 11  COD_DEPE        category
 12  COD_DEPE2       int64   
 13  RURAL_RBD       category
 14  ESTADO_ESTAB    category
 15  COD_ENSE        int64   
 16  COD_ENSE2       int64   
 17  COD_GRADO       int64   
 18  LET_CUR         category
 19  COD_JOR         int64   
 20  COD_TIP_CUR     int64   
 21  COD_DES_CUR     int64   
 22  MRUN            int64   
 23  GEN_ALU         category
 24  FEC_NAC_ALU     int64   
 25  EDAD_ALU        int64   
 26  COD_REG_ALU     int64   
 27  COD_COM_ALU 

El dataset abarca más de 3 millones de alumnos.

Si bien hay muchas columnas, los datos que estas proveen pueden ser agrupados en menos categorías:

- Identificación del alumno: edad, género, y lugar de residencia
- Rendimiento del alumno: curso, promedio general y porcentaje de asistencia
- Identificación del colegio: identificador único (RBD), ubicación, administración (municipal, particular subvencionado, etc.).
- Tipo de enseñanza: técnico-profesional, científico-humanista, para adultos, etc.

## Cobertura

In [4]:
print(f"Hay {len(rendimiento.COD_PRO_RBD.unique())} provincias, "
    + f"{len(rendimiento.COD_COM_RBD.unique())} comunas"
    + f" y {len(rendimiento.RBD.unique())} establecimientos distintos.")

Hay 56 provincias, 345 comunas y 9040 establecimientos distintos.


Chile tiene 56 provincias y 346 comunas, es decir, hay una sola comuna para la cual no hay ningún establecimiento en los datos. Todo el resto del territorio debiese estar representado.

De una [fuente externa](https://www.bcn.cl/siit/estadisticasterritoriales//resultados-consulta?id=82491) se obtiene que en Chile existen 11342 establecimientos educacionales, de los cuales 9040 están presentes en los datos. No está claro por qué estos establecimientos están ausentes, pero es probable que sean aquellos con menor cantidad de estudiantes. En general, el dataset parece ser bastante completo.

## Datos faltantes o incompletos

In [10]:
rendimiento.isna().describe()

,AGNO,RBD,DGV_RBD,NOM_RBD,COD_REG_RBD,NOM_REG_RBD_A,COD_PRO_RBD,COD_COM_RBD,NOM_COM_RBD,COD_DEPROV_RBD,...,COD_COM_ALU,NOM_COM_ALU,COD_RAMA,COD_SEC,COD_ESPE,PROM_GRAL,ASISTENCIA,SIT_FIN,SIT_FIN_R,COD_MEN
count,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,...,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
freq,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,...,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534,3164534


Aparentemente no existen datos faltantes, pero esto se debe a que los valores faltantes o incompletos están codificados de otro modo. Por ejemplo, la edad faltante se codifica con el valor 0.

In [57]:
# situacion final (aprobado, reprobado)
print(f"Sin info. de situación final: {(rendimiento.SIT_FIN_R == 'SIN_INFO').sum()}")

# genero
print(f"Sin género: {(rendimiento.GEN_ALU == 'S/I').sum()}")

# edad
print(f"Sin edad: {(rendimiento.EDAD_ALU == 0).sum()}")

# lugar de residencia
print(f"Sin lugar: {((rendimiento.COD_REG_ALU == 0) | (rendimiento.COD_COM_ALU == 0)).sum()}")

# alumnos retirados (promedio 0)
print(f"promedio 0: {(rendimiento.PROM_GRAL == 0.0).sum()}")

Sin info. de situación final: 1197
Sin género: 5
Sin edad: 636
Sin lugar: 2
promedio 0: 115617


# Preguntas y problemas

Al examinar el dataset se hace evidente que muchas variables no tienen el poder predictivo que se esperaba. Las diferencias más marcadas se dan en relación a la ubicación geográfica y el código de dependencia. Creemos que esto se debe a que los sistemas de evaluación utilizados por los colegios no están estandarizados mucho más allá de la escala de notas y los contenidos a pasar. Hay una multiplicidad de variables que afectan la forma en que los colegios evalúan a sus alumnos, por lo que las notas son poco comparables entre alumnos de colegios distintos. En consecuencia, es evidente que es necesario añadir datos que sean comparables entre colegios, o bien, derivar tales variables a partir de los atributos ya existentes. Por ejemplo, se puede considerar la distribución de notas de los colegios y tomar en cuenta la posición del alumno en relación a sus compañeros. 

Es necesario, entonces, buscar relaciones más complejas entre las variables. Al comparar promedios una variable a la vez se hace difícil observar tendencias. Por tanto, nos interesa utilizar algoritmos de clustering para determinar si existen combinaciones de variables que generen grupos de alumnos con rendimiento escolar similar. En general, del análisis de datos como los resultados de la PSU se observa que en Chile existe esta segregación, y nos interesa determinar si es posible identificarlos con los datos disponibles en este dataset.

# Propuesta experimental

Esto es parte del hito 2. Proponer y justificar los experimentos para el hito 3.

## Implementación

Implementar al menos uno de los experimentos propuestos.